# Ajustar un modelo pre-entrenado para la clasifición de textos

Para hacer el algoritmo supervisado se usará un modelo pre-entrenado, en concreto BERT, pues es uno de los mas usados para trabajar con textos en ingles.

Tambien se hará un Fine-tuning para usar el modelo pre-entrenado y entrenarlo sobre el dataset.


Instalamos las librerías que vamos a necesitar:

# Cargando Librerias

In [2]:
import pandas as pd
import numpy as np

# Cargando Datos



In [3]:
from google.colab import drive
# mount your google drive
drive.mount('/content/drive')
PATH = "/content/drive/My Drive/NLP/Proyecto/"

df_polarity = pd.read_csv(PATH+ "data_polarity.csv") 
df_country = pd.read_csv(PATH+ "data_country.csv") 
df_country_1 = pd.read_csv(PATH+ "data_country_otro_modelo.csv") 
df_test = pd.read_csv(PATH+ "data_type.csv") 
df_type = pd.read_csv(PATH+ "data_type_otro_modelo.csv") 
print('dataset cargado:', df_polarity.shape)
df_polarity.head()

df_polarity_2 = pd.read_csv(PATH+ "data_polarity_same_clases_size.csv") 
df_polarity_3 = pd.read_csv(PATH+ "data_polarity_same_clases_size_twitter.csv") 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset cargado: (107863, 4)


Los valores de type se probo con 2 dos modelos, el que se seleeciono fue poniendo las clases balanceadas y usadno berto_sentiment


los valores de type se probaron dosmodelos ambos dieron valores de las metricas parecidos. (Hacer un analisis a ver enque valores se diferenica)

los valores de country se probaorn con dos modelos con el de berto dio mejores resultados


# Cargando datos

In [4]:
df_polarity_p = df_polarity['polarity'].apply(lambda x: x+1)
df_polarity_p

0         5
1         5
2         3
3         5
4         5
         ..
107858    5
107859    5
107860    5
107861    5
107862    5
Name: polarity, Length: 107863, dtype: int64

In [5]:
df_country_c = df_country['country'].apply(lambda x: 'Colombia' if x == 0 else 'Cuba' if x == 1 else 'Mexico')
df_country_c

0           Mexico
1             Cuba
2             Cuba
3             Cuba
4           Mexico
            ...   
107858    Colombia
107859        Cuba
107860        Cuba
107861      Mexico
107862      Mexico
Name: country, Length: 107863, dtype: object

In [6]:
df_country['country']

0         2
1         1
2         1
3         1
4         2
         ..
107858    0
107859    1
107860    1
107861    2
107862    2
Name: country, Length: 107863, dtype: int64

In [7]:
df_type_t = df_test['type'].apply(lambda x: 'Attractive' if x == 0 else 'Hotel' if x == 1 else 'Restaurant')
df_type_t

0              Hotel
1              Hotel
2         Attractive
3         Attractive
4         Attractive
             ...    
107858         Hotel
107859         Hotel
107860         Hotel
107861    Attractive
107862         Hotel
Name: type, Length: 107863, dtype: object

In [8]:
df_test['type']

0         1
1         1
2         0
3         0
4         0
         ..
107858    1
107859    1
107860    1
107861    0
107862    1
Name: type, Length: 107863, dtype: int64

In [9]:
df_polarity_p_2 = df_polarity_2['polarity'].apply(lambda x: x+1)
df_polarity_p_2

0         4
1         5
2         3
3         4
4         5
         ..
107858    5
107859    4
107860    4
107861    5
107862    5
Name: polarity, Length: 107863, dtype: int64

In [10]:
df_polarity_p_3 = df_polarity_3['polarity'].apply(lambda x: x+1)
df_polarity_p_3

0         4
1         5
2         3
3         4
4         5
         ..
107858    4
107859    4
107860    4
107861    1
107862    5
Name: polarity, Length: 107863, dtype: int64

In [11]:
df_type_otro = df_type['type'].apply(lambda x: 'Attractive' if x == 0 else 'Hotel' if x == 1 else 'Restaurant')
df_type_otro

0              Hotel
1              Hotel
2         Attractive
3         Attractive
4         Attractive
             ...    
107858         Hotel
107859         Hotel
107860         Hotel
107861    Attractive
107862    Restaurant
Name: type, Length: 107863, dtype: object

In [12]:
df_country_otro = df_country_1['country'].apply(lambda x: 'Colombia' if x == 0 else 'Cuba' if x == 1 else 'Mexico')
df_country_otro

0           Mexico
1             Cuba
2           Mexico
3             Cuba
4           Mexico
            ...   
107858    Colombia
107859    Colombia
107860        Cuba
107861      Mexico
107862      Mexico
Name: country, Length: 107863, dtype: object

# Todos juntos

In [13]:
df_result = df_polarity
df_result['polarity1'] = df_polarity_p
df_result['polarity2'] = df_polarity_p_2
df_result['polarity'] = df_polarity_p_3
df_result['country'] = df_country_c
df_result['type'] = df_type_t
df_result['country_otro'] = df_country_otro
df_result['type_otro'] = df_type_otro
df_result

,ID,Title,Review,polarity,polarity1,polarity2,country,type,country_otro,type_otro
0,0,"Excelente servicio, buenas instalaciones, limp...","Mi estancia fue tan sólo de 2 días, el precio ...",4,5,4,Mexico,Hotel,Mexico,Hotel
1,1,Lo pasamos genial,Pasamos unos días muy agradables en el memorie...,5,5,5,Cuba,Hotel,Cuba,Hotel
2,2,Muy congestionado,Demasiada gente en un vagón aunque es un medio...,3,3,3,Cuba,Attractive,Mexico,Attractive
3,3,Fantastica,zona restaurada. Hay una cervecería artesanal ...,4,5,4,Cuba,Attractive,Cuba,Attractive
4,4,Por algo es una de las maravillas del mundo,Visite la zona arqueológica en compañía de mi ...,5,5,5,Mexico,Attractive,Mexico,Attractive
...,...,...,...,...,...,...,...,...,...,...
107858,107858,Modesto Cali,La pasamos muy bien restaurantes limpios y los...,4,5,5,Colombia,Hotel,Colombia,Hotel
107859,107859,GUAU,"Un gran complejo turístico todo incluido, con ...",4,5,4,Cuba,Hotel,Colombia,Hotel
107860,107860,Complejo mejor en el que he estado en Puerto V...,"La Estancia es hermosa, la piscina y las insta...",4,5,4,Cuba,Hotel,Cuba,Hotel
107861,107861,"""Hermoso teatro Juárez en Guanajuato""","""Me encanta visitar este lugar, está ves tuve ...",1,5,5,Mexico,Attractive,Mexico,Attractive


In [14]:
df_result.iloc[107862]['Review']

'Mi marido y yo vinimos aquí durante nuestra visita en diciembre. Regresamos en nuestro segundo viaje a Nuevo Vallarta. No nos decepcionó. Miguel era nuestro servidor. Probé el jamón serrano español Chili margarita por primera vez. Si te gusta una margarita con una patada, este es tu hotel! Sabor único que sin duda incorporar en mi futuro margaritas en casa. Tan sólo nosotros dos, no tomar el barco de camarones, pero el camarón" en canoa. Puedes elegir tres estilos diferentes de gambas ¡ñam ñam! Recomendamos encarecidamente el arriero de estilo. Miguel se acordaban de nosotros desde nuestra última visita hace 6 meses! Conocer a su servidor, son impresionantes. Nos vuelven por el gran servicio, bebidas y mariscos. Esperamos que ustedes de nuevo en diciembre :-).'

In [15]:
df_result.shape[0]

107863

In [16]:
def comparar_2_columnas(df, columna1, columna2):
  result = []
  total = df.shape[0]
  for i in range(0, total):
    if(df.iloc[i][columna1] != df.iloc[i][columna2]):
      temp = [df.iloc[i]['Review'], df.loc[i][columna1], df.iloc[i][columna2]]
      result.append(temp)

  return pd.DataFrame(result)

In [17]:
a = comparar_2_columnas(df_result, 'country', 'country_otro')

In [18]:
a.shape[0]*100/df_result.shape[0]

13.65713914873497

In [69]:
a[500:600]

,0,1,2
500,Pedimos una reserva para celebrar el cumpleaño...,Colombia,Cuba
501,"es una construcción maravillosa, llena de hist...",Mexico,Colombia
502,La comida estaba muy rica. El servicio es supe...,Cuba,Colombia
503,Espectacular increíble bellísimo vale cada esf...,Mexico,Colombia
504,"2 días el hotel se quedo sin agua, ni había pa...",Mexico,Cuba
...,...,...,...
595,"Es una obra arquitectónica muy bonita, pero la...",Mexico,Cuba
596,"El mejor lugar para cenar, disfrutar de la mej...",Cuba,Colombia
597,Fui en luna de miel con mi esposo y la pasamos...,Cuba,Colombia
598,"El parque está muy padre, tiene de todo para r...",Colombia,Mexico


In [75]:
a.iloc[500][0]

'Pedimos una reserva para celebrar el cumpleaños de mi hija por la página directa del hotel Hyatt y terminamos en el restaurante  de huéspedes del hotel comiendo pizza! La respuesta de la hostess fue que el restaurante japonés estaba lleno y que automáticamente direcciona al...Más'

In [20]:
b = comparar_2_columnas(df_result, 'type', 'type_otro')

In [21]:
b.shape[0]*100/df_result.shape[0]

2.070218703355182

In [50]:
b[100:200]

,0,1,2
100,"Todo estuvo organizado, limpio, muy amables de...",Attractive,Hotel
101,Yo estaba allí ayer para el día de la independ...,Attractive,Restaurant
102,La playa está muy bonita y puedes pasar tiempo...,Hotel,Attractive
103,"Para pasar un largo rato y disfrutar la vista,...",Attractive,Hotel
104,"Muy buen servicio, excelente información. Una ...",Restaurant,Attractive
...,...,...,...
195,Es un hotel pequeño bellamente remodelado con ...,Hotel,Restaurant
196,"Me gustó el ambiente, en el sitio hay mucha br...",Restaurant,Attractive
197,Me lo recomendó el guía que no hizo el tour po...,Attractive,Restaurant
198,pero junto ahí están los mejores tacos de coch...,Restaurant,Attractive


In [52]:
b.iloc[101][0]

'Yo estaba allí ayer para el día de la independencia de Colombia. Me decidí a ir muy temprano (6:45 a.m.), ya que era una fiesta, pensamos que estaría lleno. Que era perfecto, compramos nuestros billetes para el ascensor y se fue sin problemas. Las vistas desde el serro son impresionantes. Hay muchos recuerdos a elegir. . . Sin embargo, la comida fue un poco difícil elegir, tamales no son las mejores. Es mejor para usted para huevos, carnes y otras cosas. Debe ir siempre en Bogotá.'

In [24]:
c = comparar_2_columnas(df_result, 'polarity2', 'polarity')

In [25]:
c.shape[0]*100/df_result.shape[0]

23.414887403465507

In [41]:
c[c[1] + 1 < c[2]]

,0,1,2
82,Estuve alojada en este hotel 2 noches en novie...,2,4
83,El hotel está perfectamente situado en el cent...,2,4
97,"Niños corriendo, novios paseando, familias de ...",3,5
105,Es una situación simplemente increíble. Como s...,3,5
140,exelente pedi cabrito y machitos y veneno exel...,1,5
...,...,...,...
25166,Con Trump perder una pierna con sus casinos y ...,1,5
25181,"Lugar muy bonito, te imaginas otra cosa es muy...",3,5
25191,"En general, tuvimos una estancia excelente! En...",2,4
25207,Estupendas instalaciones! Súper actitud de tod...,2,4


In [46]:
c.iloc[25191][0]

'En general, tuvimos una estancia excelente! En el terreno de la propiedad, sin embargo, hermosa la mejor parte de nuestra estancia fue el servicio que recibimos. Sobre todo en cuanto a viajar con un bebé. El hotel superó nuestras expectativas con tratando de acomodar de alguna manera con ella. Botella de lavado, cuna, sillas altas y un servicio amable en todos los lugares de comidas. Nos gustaría reconocer conserje Adrian, piscina camareros Nadia y Jacqueline. Simplemente fantástico en todos los aspectos de hacer nuestra estancia inolvidable.\n\nSólo cenamos una vez en Lucca, y aprovechamos el servicio de habitación cada noche como ya se ha dicho que viaja con un bebé. Lucca fue genial, pasamos la lasaña y la lubina. El servicio de habitación era siempre caliente y fresco. Sinceramente el servicio de habitación las 24 horas no puede ser mejor.\n\nLa silla de piscina ahorrando es molesto, pero el personal hace un trabajo bastante bueno de encontrar una silla si necesitas uno.\n\nY siem

In [ ]:
df_result = df_result.drop(['polarity1', 'polarity2', 'country', 'type'], axis = 1)
df_result

,ID,Title,Review,polarity,country_otro,type_otro
0,0,"Excelente servicio, buenas instalaciones, limp...","Mi estancia fue tan sólo de 2 días, el precio ...",4,Mexico,Hotel
1,1,Lo pasamos genial,Pasamos unos días muy agradables en el memorie...,5,Cuba,Hotel
2,2,Muy congestionado,Demasiada gente en un vagón aunque es un medio...,3,Mexico,Attractive
3,3,Fantastica,zona restaurada. Hay una cervecería artesanal ...,4,Cuba,Attractive
4,4,Por algo es una de las maravillas del mundo,Visite la zona arqueológica en compañía de mi ...,5,Mexico,Attractive
...,...,...,...,...,...,...
107858,107858,Modesto Cali,La pasamos muy bien restaurantes limpios y los...,4,Colombia,Hotel
107859,107859,GUAU,"Un gran complejo turístico todo incluido, con ...",4,Colombia,Hotel
107860,107860,Complejo mejor en el que he estado en Puerto V...,"La Estancia es hermosa, la piscina y las insta...",4,Cuba,Hotel
107861,107861,"""Hermoso teatro Juárez en Guanajuato""","""Me encanta visitar este lugar, está ves tuve ...",1,Mexico,Attractive


In [ ]:
print(df_result.shape)

(107863, 6)


In [ ]:
df_result.to_csv(PATH + 'data_final.csv', index=False, encoding='utf-8-sig')

# Creando txt

In [ ]:
with open(PATH + 'result_output.txt', 'w') as f:
  for index, row in df_result.iterrows():
    f.write('sentiment\t' + str(row[0]) + '\t' + str(row[3]) + '\t' + row[5] + '\t' + row[4] + '\n')
    